In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 27 11:33:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    53W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00


In [ ]:
import os
import time
import librosa
import numpy as np
import pandas as pd
import sklearn
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2FeatureExtractor
import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/data1.csv', encoding = 'utf-8')
data2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/data2.csv', encoding = 'utf-8')
data3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/data3.csv', encoding = 'utf-8')
data4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/data4.csv', encoding = 'utf-8')
data5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/data5.csv', encoding = 'utf-8')

In [ ]:
df1 = data1.copy()
df2 = data2.copy()
df3 = data3.copy()
df4 = data4.copy()
df5 = data5.copy()

In [ ]:
df1.head()

,Wav_start,Wav_end,Speaker,Emotion,Valence,Arousal,IBI_mean,IBI_count,IBI_std,IBI_max,...,TEMP_min,TEMP_range,TEMP_difference,Emotion_p_i,Fleiss_Kappa,Session,Wav_length,Year,Position,path
0,15.476,19.786,Sess01_script01_F001,fear,1.5,3.6,0.940800,5.0,0.022163,0.960,...,30.69,0.02,0.02,0.488889,0.194631,Session01_F_res,4.310,19,Speaker,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
1,28.943,32.167,Sess01_script01_F002,fear,1.6,3.7,0.916000,3.0,0.024331,0.944,...,30.69,0.02,0.02,0.644444,0.198791,Session01_F_res,3.224,19,Speaker,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
2,39.264,46.833,Sess01_script01_F003,fear,1.4,4.1,0.828889,9.0,0.026816,0.868,...,30.69,0.06,0.04,0.355556,0.198791,Session01_F_res,7.569,19,Speaker,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
3,73.016,83.209,Sess01_script01_F005,fear,1.5,3.5,0.804923,13.0,0.054262,0.872,...,30.71,0.02,0.02,0.533333,0.070443,Session01_F_res,10.193,19,Speaker,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
4,97.483,105.530,Sess01_script01_F006,fear,1.4,4.0,0.813778,9.0,0.024748,0.848,...,30.71,0.06,0.02,0.488889,0.218391,Session01_F_res,8.047,19,Speaker,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...


In [ ]:
df1.columns

Index(['Wav_start', 'Wav_end', 'Speaker', 'Emotion', 'Valence', 'Arousal',
       'IBI_mean', 'IBI_count', 'IBI_std', 'IBI_max', 'IBI_min', 'IBI_range',
       'IBI_difference', 'EDA_mean', 'EDA_count', 'EDA_std', 'EDA_max',
       'EDA_min', 'EDA_range', 'EDA_difference', 'TEMP_mean', 'TEMP_count',
       'TEMP_std', 'TEMP_max', 'TEMP_min', 'TEMP_range', 'TEMP_difference',
       'Emotion_p_i', 'Fleiss_Kappa', 'Session', 'Wav_length', 'Year',
       'Position', 'path'],
      dtype='object')

In [ ]:
df1 = df1.drop(['Wav_start', 'Wav_end', 'Speaker', 'Valence', 'Arousal',
       'IBI_mean', 'IBI_count', 'IBI_std', 'IBI_max', 'IBI_min', 'IBI_range',
       'IBI_difference', 'EDA_mean', 'EDA_count', 'EDA_std', 'EDA_max',
       'EDA_min', 'EDA_range', 'EDA_difference', 'TEMP_mean', 'TEMP_count',
       'TEMP_std', 'TEMP_max', 'TEMP_min', 'TEMP_range', 'TEMP_difference',
       'Emotion_p_i', 'Fleiss_Kappa', 'Session', 'Wav_length', 'Year',
       'Position'],axis=1)
df2 = df2.drop(['Wav_start', 'Wav_end', 'Speaker', 'Valence', 'Arousal',
       'IBI_mean', 'IBI_count', 'IBI_std', 'IBI_max', 'IBI_min', 'IBI_range',
       'IBI_difference', 'EDA_mean', 'EDA_count', 'EDA_std', 'EDA_max',
       'EDA_min', 'EDA_range', 'EDA_difference', 'TEMP_mean', 'TEMP_count',
       'TEMP_std', 'TEMP_max', 'TEMP_min', 'TEMP_range', 'TEMP_difference',
       'Emotion_p_i', 'Fleiss_Kappa', 'Session', 'Wav_length', 'Year',
       'Position'],axis=1)
df3 = df3.drop(['Wav_start', 'Wav_end', 'Speaker', 'Valence', 'Arousal',
       'IBI_mean', 'IBI_count', 'IBI_std', 'IBI_max', 'IBI_min', 'IBI_range',
       'IBI_difference', 'EDA_mean', 'EDA_count', 'EDA_std', 'EDA_max',
       'EDA_min', 'EDA_range', 'EDA_difference', 'TEMP_mean', 'TEMP_count',
       'TEMP_std', 'TEMP_max', 'TEMP_min', 'TEMP_range', 'TEMP_difference',
       'Emotion_p_i', 'Fleiss_Kappa', 'Session', 'Wav_length', 'Year',
       'Position'],axis=1)
df4 = df4.drop(['Wav_start', 'Wav_end', 'Speaker', 'Valence', 'Arousal',
       'IBI_mean', 'IBI_count', 'IBI_std', 'IBI_max', 'IBI_min', 'IBI_range',
       'IBI_difference', 'EDA_mean', 'EDA_count', 'EDA_std', 'EDA_max',
       'EDA_min', 'EDA_range', 'EDA_difference', 'TEMP_mean', 'TEMP_count',
       'TEMP_std', 'TEMP_max', 'TEMP_min', 'TEMP_range', 'TEMP_difference',
       'Emotion_p_i', 'Fleiss_Kappa', 'Session', 'Wav_length', 'Year',
       'Position'],axis=1)
df5 = df5.drop(['Wav_start', 'Wav_end', 'Speaker', 'Valence', 'Arousal',
       'IBI_mean', 'IBI_count', 'IBI_std', 'IBI_max', 'IBI_min', 'IBI_range',
       'IBI_difference', 'EDA_mean', 'EDA_count', 'EDA_std', 'EDA_max',
       'EDA_min', 'EDA_range', 'EDA_difference', 'TEMP_mean', 'TEMP_count',
       'TEMP_std', 'TEMP_max', 'TEMP_min', 'TEMP_range', 'TEMP_difference',
       'Emotion_p_i', 'Fleiss_Kappa', 'Session', 'Wav_length', 'Year',
       'Position'],axis=1)

In [ ]:
df1

,Emotion,path
0,fear,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
1,fear,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
2,fear,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
3,fear,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
4,fear,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
...,...,...
3055,neutral,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
3056,neutral,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
3057,neutral,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...
3058,neutral,/content/drive/MyDrive/Colab Notebooks/2023 휴먼...


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
df_train1 = pd.concat([df2, df3, df4, df5]).copy().reset_index(drop=True)
df_test1 = df1.copy()
df_train2 = pd.concat([df1, df3, df4, df5]).copy().reset_index(drop=True)
df_test2 = df2.copy()
df_train3 = pd.concat([df1, df2, df4, df5]).copy().reset_index(drop=True)
df_test3 = df3.copy()
df_train4 = pd.concat([df1, df2, df3, df5]).copy().reset_index(drop=True)
df_test4 = df4.copy()
df_train5 = pd.concat([df1, df2, df3, df4]).copy().reset_index(drop=True)
df_test5 = df5.copy()

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
save_path = "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/"

In [ ]:
print(df_train1.shape)
print(df_test1.shape)

(12238, 2)
(3060, 2)


In [ ]:
df_train1.to_csv(f"{save_path}/train1.csv", sep=",", encoding="utf-8", index=False)
df_test1.to_csv(f"{save_path}/test1.csv", sep=",", encoding="utf-8", index=False)

df_train2.to_csv(f"{save_path}/train2.csv", sep=",", encoding="utf-8", index=False)
df_test2.to_csv(f"{save_path}/test2.csv", sep=",", encoding="utf-8", index=False)

df_train3.to_csv(f"{save_path}/train3.csv", sep=",", encoding="utf-8", index=False)
df_test3.to_csv(f"{save_path}/test3.csv", sep=",", encoding="utf-8", index=False)

df_train4.to_csv(f"{save_path}/train4.csv", sep=",", encoding="utf-8", index=False)
df_test4.to_csv(f"{save_path}/test4.csv", sep=",", encoding="utf-8", index=False)

df_train5.to_csv(f"{save_path}/train5.csv", sep=",", encoding="utf-8", index=False)
df_test5.to_csv(f"{save_path}/test5.csv", sep=",", encoding="utf-8", index=False)

In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    CUT = 10 # custom cut at 4 seconds for speeding up the data processing (not necessary)
    if len(speech) > 16000*CUT:
        return speech[:int(16000*CUT)]
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
input_column = "path"
output_column = "Emotion"

label_list = train_dataset5.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 7 classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
model_name_or_path = "kresnik/wav2vec2-large-xlsr-korean"
pooling_mode = "mean"

In [ ]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The target sampling rate: 16000


# 모델 구성

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        
        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
is_regression = False

In [ ]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [ ]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        model.train()
        inputs = self._prepare_inputs(inputs)
        
        loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        #if self.use_amp:
            #self.scaler.scale(loss).backward()
        if self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()

In [ ]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
      model_name_or_path,
      config=config
    )

Some weights of the model checkpoint at kresnik/wav2vec2-large-xlsr-korean were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at kresnik/wav2vec2-large-xlsr-korean and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream tas

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/SER",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    #optim = 'sgd',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs = 10,
    #fp16=True,
    eval_steps=500,
    save_steps = 10000,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)

In [ ]:
from datasets import Dataset

In [ ]:
from transformers import Trainer

# 모델5 (subset5가 test)

In [ ]:
data_files5 = {
    "train":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/train5.csv", 
    "test":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/test5.csv",
}

dataset5 = load_dataset("csv", data_files=data_files5, delimiter="," ,)
train_dataset5 = dataset5["train"]
eval_dataset5 = dataset5["test"]
test_dataset5 = dataset5["test"]

print(train_dataset5)
print(eval_dataset5)
print(test_dataset5)

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Emotion', 'path'],
    num_rows: 12239
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3059
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3059
})


In [ ]:
train_dataset5 = train_dataset5.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)
eval_dataset5 = eval_dataset5.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/12239 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

Map (num_proc=8):   0%|          | 0/3059 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset5,
    eval_dataset=eval_dataset5,
    tokenizer=processor.feature_extractor,
)

In [ ]:
history = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.657500,1.445414,0.753841
1000,0.327900,1.589486,0.765937
1500,0.305700,1.974220,0.756783
2000,0.144800,2.230254,0.757110
2500,0.141800,2.349453,0.757110
3000,0.099200,2.565767,0.754495
3500,0.067000,2.660978,0.754168


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# 모델4 (subset4가 test)

In [ ]:
data_files4 = {
    "train":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/train4.csv", 
    "test":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/test4.csv",
}

dataset4 = load_dataset("csv", data_files=data_files4, delimiter="," ,)
train_dataset4 = dataset4["train"]
eval_dataset4 = dataset4["test"]
test_dataset4 = dataset4["test"]

print(train_dataset4)
print(eval_dataset4)
print(test_dataset4)

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Emotion', 'path'],
    num_rows: 12239
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3059
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3059
})


In [ ]:
train_dataset4 = train_dataset4.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)
eval_dataset4 = eval_dataset4.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/12239 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

Map (num_proc=8):   0%|          | 0/3059 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset4,
    eval_dataset=eval_dataset4,
    tokenizer=processor.feature_extractor,
)

In [ ]:
history = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.786300,0.687014,0.770840
1000,0.419000,0.742936,0.769859
1500,0.348800,0.909293,0.759725
2000,0.147700,1.039213,0.768552
2500,0.085200,1.177862,0.761360
3000,0.114900,1.324601,0.759072
3500,0.045600,1.388814,0.758091


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# 모델3 (subset3가 test)

In [ ]:
data_files3 = {
    "train":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/train3.csv", 
    "test":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/test3.csv",
}

dataset3 = load_dataset("csv", data_files=data_files3, delimiter="," ,)
train_dataset3 = dataset3["train"]
eval_dataset3 = dataset3["test"]
test_dataset3 = dataset3["test"]

print(train_dataset3)
print(eval_dataset3)
print(test_dataset3)

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Emotion', 'path'],
    num_rows: 12238
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3060
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3060
})


In [ ]:
train_dataset3 = train_dataset3.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)
eval_dataset3 = eval_dataset3.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/12238 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

Map (num_proc=8):   0%|          | 0/3060 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset3,
    eval_dataset=eval_dataset3,
    tokenizer=processor.feature_extractor,
)

In [ ]:
history = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.662900,0.794469,0.747712
1000,0.375300,0.827216,0.761765
1500,0.289600,0.954747,0.745098
2000,0.208900,1.085263,0.758497
2500,0.100400,1.225419,0.739542
3000,0.081800,1.300375,0.750980
3500,0.137200,1.368234,0.753922


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# 모델2 (subset2가 test)

In [ ]:
data_files2 = {
    "train":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/train2.csv", 
    "test":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/test2.csv",
}

dataset2 = load_dataset("csv", data_files=data_files2, delimiter="," ,)
train_dataset2 = dataset2["train"]
eval_dataset2 = dataset2["test"]
test_dataset2 = dataset2["test"]

print(train_dataset2)
print(eval_dataset2)
print(test_dataset2)

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Emotion', 'path'],
    num_rows: 12238
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3060
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3060
})


In [ ]:
train_dataset2 = train_dataset2.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)
eval_dataset2 = eval_dataset2.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/12238 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

Map (num_proc=8):   0%|          | 0/3060 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset2,
    eval_dataset=eval_dataset2,
    tokenizer=processor.feature_extractor,
)

In [ ]:
history = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.637800,0.911360,0.695752
1000,0.407000,0.838702,0.751634
1500,0.256400,0.968439,0.739216
2000,0.179400,1.189059,0.709477
2500,0.161900,1.350600,0.743137
3000,0.106800,1.284343,0.746078
3500,0.099100,1.417442,0.740196


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# 모델1 (subset1이 test)

In [ ]:
data_files1 = {
    "train":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/train1.csv", 
    "test":  "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/test1.csv",
}

dataset1 = load_dataset("csv", data_files=data_files1, delimiter="," ,)
train_dataset1 = dataset1["train"]
#eval_dataset1 = dataset1["test"]
test_dataset1 = dataset1["test"]

print(train_dataset1)
#print(eval_dataset1)
print(test_dataset1)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3df8e423b937b797/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Emotion', 'path'],
    num_rows: 12238
})
Dataset({
    features: ['Emotion', 'path'],
    num_rows: 3060
})


In [ ]:
eval_dataset1 = dataset1["test"]

In [ ]:
train_dataset1 = train_dataset1.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)
eval_dataset1 = eval_dataset1.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/12238 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
train_dataset1

Dataset({
    features: ['Emotion', 'path', 'input_values', 'attention_mask', 'labels'],
    num_rows: 12238
})

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset1,
    eval_dataset=eval_dataset1,
    tokenizer=processor.feature_extractor,
)

In [ ]:
history = trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.555900,1.146967,0.596405
1000,0.347400,1.254667,0.603268
1500,0.228600,1.280700,0.635294
2000,0.199300,1.503513,0.638889
2500,0.112000,1.638695,0.634967
3000,0.087100,1.902264,0.633333
3500,0.069100,1.790961,0.655556


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# test

In [ ]:
model

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/model5"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [ ]:
model.save_pretrained(model_path)
config.save_pretrained(model_path)
processor.save_pretrained(model_path)

In [ ]:
config = AutoConfig.from_pretrained(model_path, local_files_only=True)
processor = Wav2Vec2Processor.from_pretrained(model_path, local_files_only=True)
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_path, local_files_only=True).to(device)

In [ ]:
model.cuda()

In [ ]:
def logit(batch):

    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
    
    logits = F.softmax(logits, dim=1).detach().cpu().numpy()
    
    batch['logits'] = logits
    return batch

In [ ]:
test_df5 = test_dataset5.to_pandas()
test_df5['speech'] = test_df5['path'].map(speech_file_to_array_fn)
test_dataset5 = Dataset.from_pandas(test_df5)
test_dataset5 = test_dataset5.map(logit, batched = True, batch_size=1)
test_df5 = test_dataset5.to_pandas()
test_df5 = test_df5[['Emotion','path','logits']]
test_df5.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/late/late5.pkl')

Map:   0%|          | 0/3059 [00:00<?, ? examples/s]

In [ ]:
test_df4 = test_dataset4.to_pandas()
test_df4['speech'] = test_df4['path'].map(speech_file_to_array_fn)
test_dataset4 = Dataset.from_pandas(test_df4)
test_dataset4 = test_dataset4.map(logit, batched = True, batch_size=1)
test_df4 = test_dataset4.to_pandas()
test_df4 = test_df4[['Emotion','path','logits']]
test_df4.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/late/late4.pkl')

Map:   0%|          | 0/3059 [00:00<?, ? examples/s]

In [ ]:
test_df3 = test_dataset3.to_pandas()
test_df3['speech'] = test_df3['path'].map(speech_file_to_array_fn)
test_dataset3 = Dataset.from_pandas(test_df3)
test_dataset3 = test_dataset3.map(logit, batched = True, batch_size=1)
test_df3 = test_dataset3.to_pandas()
test_df3 = test_df3[['Emotion','path','logits']]
test_df3.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/late/late3.pkl')

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
test_df2 = test_dataset2.to_pandas()
test_df2['speech'] = test_df2['path'].map(speech_file_to_array_fn)
test_dataset2 = Dataset.from_pandas(test_df2)
test_dataset2 = test_dataset2.map(logit, batched = True, batch_size=1)
test_df2 = test_dataset2.to_pandas()
test_df2 = test_df2[['Emotion','path','logits']]
test_df2.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/late/late1.pkl')

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
test_df1 = test_dataset1.to_pandas()
test_df1['speech'] = test_df1['path'].map(speech_file_to_array_fn)
test_dataset1 = Dataset.from_pandas(test_df1)
test_dataset1 = test_dataset1.map(logit, batched = True, batch_size=1)
test_df1 = test_dataset1.to_pandas()
test_df1 = test_df1[['Emotion','path','logits']]
test_df1.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/late/late1.pkl')

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
def feature_vector(batch):

    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        feature_vector = model(input_values, attention_mask=attention_mask).hidden_states 
    feature_vector = feature_vector.detach().cpu().numpy()
    batch['feature'] = feature_vector
    return batch

In [ ]:
train_df5 = train_dataset5.to_pandas()
train_df5['speech'] = train_df5['path'].map(speech_file_to_array_fn)
train_dataset5 = Dataset.from_pandas(train_df5)
train_dataset5 = train_dataset5.map(feature_vector, batched = True, batch_size=1)
train_df5 = train_dataset5.to_pandas()
train_df5 = train_df5[['Emotion','path','feature']]
train_df5.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/train5.pkl')

test_df5 = test_dataset5.to_pandas()
test_df5['speech'] = test_df5['path'].map(speech_file_to_array_fn)
test_dataset5 = Dataset.from_pandas(test_df5)
test_dataset5 = test_dataset5.map(feature_vector, batched = True, batch_size=1)
test_df5 = test_dataset5.to_pandas()
test_df5 = test_df5[['Emotion','path','feature']]
test_df5.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/test5.pkl')

Map:   0%|          | 0/12239 [00:00<?, ? examples/s]

Map:   0%|          | 0/3059 [00:00<?, ? examples/s]

In [ ]:
train_df4 = train_dataset4.to_pandas()
train_df4['speech'] = train_df4['path'].map(speech_file_to_array_fn)
train_dataset4 = Dataset.from_pandas(train_df4)
train_dataset4 = train_dataset4.map(feature_vector, batched = True, batch_size=1)
train_df4 = train_dataset4.to_pandas()
train_df4 = train_df4[['Emotion','path','feature']]
train_df4.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/train4.pkl')

test_df4 = test_dataset4.to_pandas()
test_df4['speech'] = test_df4['path'].map(speech_file_to_array_fn)
test_dataset4 = Dataset.from_pandas(test_df4)
test_dataset4 = test_dataset4.map(feature_vector, batched = True, batch_size=1)
test_df4 = test_dataset4.to_pandas()
test_df4 = test_df4[['Emotion','path','feature']]
test_df4.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/test4.pkl')

Map:   0%|          | 0/12239 [00:00<?, ? examples/s]

Map:   0%|          | 0/3059 [00:00<?, ? examples/s]

In [ ]:
train_df3 = train_dataset3.to_pandas()
train_df3['speech'] = train_df3['path'].map(speech_file_to_array_fn)
train_dataset3 = Dataset.from_pandas(train_df3)
train_dataset3 = train_dataset3.map(feature_vector, batched = True, batch_size=1)
train_df3 = train_dataset3.to_pandas()
train_df3 = train_df3[['Emotion','path','feature']]
train_df3.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/train3.pkl')

test_df3 = test_dataset3.to_pandas()
test_df3['speech'] = test_df3['path'].map(speech_file_to_array_fn)
test_dataset3 = Dataset.from_pandas(test_df3)
test_dataset3 = test_dataset3.map(feature_vector, batched = True, batch_size=1)
test_df3 = test_dataset3.to_pandas()
test_df3 = test_df3[['Emotion','path','feature']]
test_df3.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/test3.pkl')

Map:   0%|          | 0/12238 [00:00<?, ? examples/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
train_df2 = train_dataset2.to_pandas()
train_df2['speech'] = train_df2['path'].map(speech_file_to_array_fn)
train_dataset2 = Dataset.from_pandas(train_df2)
train_dataset2 = train_dataset2.map(feature_vector, batched = True, batch_size=1)
train_df2 = train_dataset2.to_pandas()
train_df2 = train_df2[['Emotion','path','feature']]
train_df2.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/train2.pkl')

test_df2 = test_dataset2.to_pandas()
test_df2['speech'] = test_df2['path'].map(speech_file_to_array_fn)
test_dataset2 = Dataset.from_pandas(test_df2)
test_dataset2 = test_dataset2.map(feature_vector, batched = True, batch_size=1)
test_df2 = test_dataset2.to_pandas()
test_df2 = test_df2[['Emotion','path','feature']]
test_df2.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/test2.pkl')

Map:   0%|          | 0/12238 [00:00<?, ? examples/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
train_df1 = train_dataset1.to_pandas()
train_df1['speech'] = train_df1['path'].map(speech_file_to_array_fn)
train_dataset1 = Dataset.from_pandas(train_df1)
train_dataset1 = train_dataset1.map(feature_vector, batched = True, batch_size=1)
train_df1 = train_dataset1.to_pandas()
train_df1 = train_df1[['Emotion','path','feature']]
train_df1.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/train1.pkl')

test_df1 = test_dataset1.to_pandas()
test_df1['speech'] = test_df1['path'].map(speech_file_to_array_fn)
test_dataset1 = Dataset.from_pandas(test_df1)
test_dataset1 = test_dataset1.map(feature_vector, batched = True, batch_size=1)
test_df1 = test_dataset1.to_pandas()
test_df1 = test_df1[['Emotion','path','feature']]
test_df1.to_pickle('/content/drive/MyDrive/Colab Notebooks/2023 휴먼이해 공모전/곽소정/data/early/test1.pkl')

Map:   0%|          | 0/12238 [00:00<?, ? examples/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
test_dataset = load_dataset("csv", data_files={"test": save_path +"test.csv"}, delimiter=",")["test"]
test_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3dcac93cc1a0a99e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Emotion', 'path'],
    num_rows: 4491
})

In [ ]:
def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)
    #print(features)
    input_values = features.input_values.to(device)
    attention_mask1 = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values).logits 

    #print(logits)
    #print(torch.argmax(logits, dim = -1).detach().cpu().numpy())
    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    #print(batch['predicted'])
    return batch

In [ ]:
test_df5 = test_dataset5.to_pandas()
test_df5['speech'] = test_df5['path'].map(speech_file_to_array_fn)
test_dataset5 = Dataset.from_pandas(test_df5)
result5 = test_dataset5.map(predict, batched=True, batch_size=1)

Map:   0%|          | 0/3059 [00:00<?, ? examples/s]

In [ ]:
test_df4 = test_dataset4.to_pandas()
test_df4['speech'] = test_df4['path'].map(speech_file_to_array_fn)
test_dataset4 = Dataset.from_pandas(test_df4)
result3 = test_dataset4.map(predict, batched=True, batch_size=1)

Map:   0%|          | 0/3059 [00:00<?, ? examples/s]

In [ ]:
test_df3 = test_dataset3.to_pandas()
test_df3['speech'] = test_df3['path'].map(speech_file_to_array_fn)
test_dataset3 = Dataset.from_pandas(test_df3)
result3 = test_dataset3.map(predict, batched=True, batch_size=1)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
test_df2 = test_dataset2.to_pandas()
test_df2['speech'] = test_df2['path'].map(speech_file_to_array_fn)
test_dataset2 = Dataset.from_pandas(test_df2)
result2 = test_dataset2.map(predict, batched=True, batch_size=1)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset

In [ ]:
test_dataset1

Dataset({
    features: ['Emotion', 'path', 'speech'],
    num_rows: 3060
})

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [ ]:
model.cuda()

In [ ]:
t_dsds = t_ds.map(predict, batched=True, batch_size=1)

In [ ]:
result = test_dataset.map(predict, batched=True)

In [ ]:
np.unique(result1['predicted'])

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
y_true = [config.label2id[name] for name in result5["Emotion"]]
y_pred = result5["predicted"]

In [ ]:
label_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [ ]:
#label_names = [config.id2label[i] for i in range(config.num_labels)]
y_true = result1["Emotion"]
y_pred = [config.id2label[num] for num in result1["predicted"]]

In [ ]:
y_true

In [ ]:
result['Emotion']

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    # plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def one_hot_encoder(true_labels, num_records, num_classes):
    temp = np.array(true_labels[:num_records])
    true_labels = np.zeros((num_records, num_classes))
    true_labels[np.arange(num_records), temp] = 1
    return true_labels

def display_results(y_test, y_pred, cm=True):
    #one_hot_true = one_hot_encoder(y_test, len(pred), len(emotion_dict))
    print('Test Set Accuracy =  {0:.4f}'.format(accuracy_score(y_test, y_pred)))
    print('Test Set F-score =  {0:.4f}'.format(f1_score(y_test, y_pred, average='macro')))
    print('Test Set Precision =  {0:.4f}'.format(precision_score(y_test, y_pred, average='macro')))
    print('Test Set Recall =  {0:.4f}'.format(recall_score(y_test, y_pred, average='macro')))
    if cm:
        plot_confusion_matrix(confusion_matrix(y_test, y_pred), classes=label_names)

In [ ]:
print('macro')
print('Test Set Accuracy =  {0:.4f}'.format(accuracy_score(y_true, y_pred)))
print('Test Set F-score =  {0:.4f}'.format(f1_score(y_true, y_pred, average='macro')))
print('Test Set Precision =  {0:.4f}'.format(precision_score(y_true, y_pred, average='macro')))
print('Test Set Recall =  {0:.4f}'.format(recall_score(y_true, y_pred, average='macro')))
print("micro")
print('Test Set Accuracy =  {0:.4f}'.format(accuracy_score(y_true, y_pred)))
print('Test Set F-score =  {0:.4f}'.format(f1_score(y_true, y_pred, average='micro')))
print('Test Set Precision =  {0:.4f}'.format(precision_score(y_true, y_pred, average='micro')))
print('Test Set Recall =  {0:.4f}'.format(recall_score(y_true, y_pred, average='micro')))
plot_confusion_matrix(confusion_matrix(y_true, y_pred), classes=label_names)